# Notebook 10 - Reinforcement Learning / Self-Driving Cab

CSI4106 Artificial Intelligence   
Fall 2021  
Version 1 (2020) prepared by Julian Templeton and Caroline Barrière.  Version 2 (2021) adapted by Caroline Barrière.

***INTRODUCTION***:  
In this notebook we will be exploring the use of Reinforcment Learning to help allow an agent solve a specific task in an environment provided by [OpenAI's Gym library](https://gym.openai.com/). OpenAI's Gym provides many different experiments to use. These range from balancing acts to self driving cars to playing a simple Atari game. Unfortunately not every option available to us can be easily worked with. Many can take hours of training to start seeing some exciting results. Each of these experiments use agents that can be trained by Reinforcment Learning to master how to perform the specified task. The methods used can range from the simple use of Q-Learning to the more complex use of one or more Deep Learning models that work in conjunction with Reinforcement Learning techniques. 

Within this notebook we will be exploring a scenario in which an autonomous taxi located on a grid must pickup a passenger located in one of four positions and drop the passenger off in one of three other positions.    

To familiarize yourself with the **Self-Driving Cab problem** tackled in this notebook, please go to the site https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/ and read:  \

*   section 1 (rewards)
*   section 2 (state space) which will make you understand why there are 500 possible states
*   section 3 (action space) which describes the possible actions.  

Throughout the notebook we will be working with a random baseline approach and a Q-Learning-based approach. This will provide insight into how Q-Learning can be applied to problems and how an agent can use Reinforcment Learning to solve problems in an environment.  Comparison with the baseline approach will show the potential of Q-Learning.  

**When submitting this notebook, ensure that you do NOT reset the outputs from running the code (plus remember to save the notebook with ctrl+s).**      

**In order to keep the installation easy, you will be once again running this notebook in Google Colab, NOT on your local machine.**    

***HOMEWORK***:  
Go through the notebook by running each cell, one at a time.  
Look for **(TO DO)** for the tasks that you need to perform. Do not edit the code outside of the questions which you are asked to answer unless specifically asked. Once you're done, sign the notebook (at the end of the notebook), rename it to *StudentNumber-LastName-Notebook10.ipynb* and submit it.  

*The notebook will be marked on 30.  
Each **(TO DO)** has a number of points associated with it.*
***

**1.0 - Setting up the Taxi Game**   

To begin the notebook, we need to set up and explore the Open Gym autonomous taxi environment.  

The autonomous taxi will pick up and dropoff a passenger within a small grid. To really understand the environment in which the autonomous taxi (the agent) evolves, make sure you read the 3 sections about rewards, state space and action space from the tutorial site, as mentioned in the introduction of this notebook.

The code used throughout the notebook comes from [this example](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/) and has been modified accordingly.

To start, we will install some of the packages that we will need to run the progam.

In [ ]:
# Install the necessary libraries
!pip install cmake 'gym[atari]' scipy

In [ ]:
# Import the necessary libraries
import random
import gym
import numpy as np
from IPython.display import clear_output

With all of the libraries installed, we will now make use of the Taxi program provided by Gym. Below we will import Gym, load the program as the active environment, and render an image representing the current state of the program.   

From the image seen below, there are four different key locations in the environment, represented by *R*, *G*, *Y*, and *B*. The letter that is *bolded in blue* represents where the current passenger needs to get picked up and the letter *bolded in purple* represents where the passenger wants to dropped off. The *yellow block* represents the cell which the taxi cab is currently located at. Therefore, the taxi cab must first pick up the passenger and drop them off at the dropoff location. When a passenger is in the taxi, the yellow block turns to a *green block* until the passenger is dropped off.

In [ ]:
# Load the environment
env = gym.make("Taxi-v3").env
# Render the current state of the program
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Next we will reset the state of the environment and re-render the current state. We also print the total number of actions available to our agent (defined as the *Action Space*) and the *State Space* which represents the state of the program (where is the cab, the passenger, pickup location and dropoff location).

In [ ]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


We want our agent to learn which action to take given a specific state. This state depends on where the taxi cab is located in relation to the passenger location and drop off location. The six possible actions that the taxi can take at a given time step are:    

Action = 0: Head south    
Action = 1: Head north    
Action = 2: Head east    
Action = 3: Head west    
Action = 4: Pickup     
Action = 5: Dropoff    

Below is an example of setting the state to a specific encoding and rendering that state.

In [ ]:
# The encoding below represents: (taxi row, taxi column, passenger location, destination index)
state = env.encode(3, 1, 2, 0) 
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



The following example showcases how a state with the passenger within the taxi can be set.

In [ ]:
# The encoding below represents: (taxi row, taxi column, passenger location, destination index)
state = env.encode(0, 1, 4, 0) 
print("State:", state)

env.s = state
env.render()

State: 36
+---------+
|R:_| : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



**(TO DO) Q1 - 4 marks**    
Now that we have seen how to set a state via an encoding, you will need to set the state to match two different descriptions below and render them.   

**(TO DO) Q1 (a) - 2 marks**    
Set the passenger to be at position G, with the passenger wanting to be dropped off at position R, and the taxi positioned at a random point on the grid (the selected position of the taxi must be selected randomly). After setting the position, render the state.   

In [ ]:
# ANSWER Q1(a) 
# remember to use random coordinates within the grid for the taxi...
import random
taxi_r = random.randint(0,4) # random integer for row of taxi
taxi_c = random.randint(0,4) # random integer for column of taxi

state_1a = env.encode(taxi_r, taxi_c, 1, 0) 
print("State:", state_1a)

env.s = state_1a
env.render()


State: 264
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


**(TO DO) Q1 (b) - 2 marks**    
Set the passenger to be in the taxi (at any position without a letter on it) and set the passenger dropoff point to be position B. After setting the position, render the state.

In [ ]:
# ANSWER Q1(b)
# Set passenger to be in the taxi at any position without a letter on it, and set drop off point to be position B
state_1b = env.encode(3, 3, 4, 3) 
print("State:", state_1b)

env.s = state_1b
env.render()

State: 379
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (Dropoff)


For every action that the taxi can take, we have a list representing the key information with respect to what will happen when an action is performed. After performing an action, the agent will receive a reward or a penalty from the environment which will then be in a different state.

Below we display a dictionary (very similar to the Reward Table we discussed in the video lecture) that contains all possible actions along with the following information within the corresponding tuples:     

(     
  The probability of taking that action,     
  The resulting state after taking that action,    
  The reward for taking that action,    
  Whether or not the program will end when performing the action   
)      

Example tuple: (1.0, 328, -1, False)

In [ ]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

Although not displayed by the code above, if the taxi is holding the passenger and is over the dropoff point, the reward for the dropoff action is 20.

**2.0 - Baseline Approach to the Taxi Game**   

To start, we will perform the simulation of the taxi cab scenario with a baseline approach that does not use Q-Learning. This approach will simply work by selecting a random available action at each time step, regardless of the current state. We will also prepare a method of playing through all frames within an episode to view how the agent controls the taxi in the scenario.

In [ ]:
def run_single_simulation_baseline(env, state, disable_prints=False):
    '''
    Given the environment and a specific state, randomly select an action for the taxi
    to perform until the goal is completed.
    '''
    if not disable_prints:
        print("Testing for simulation: {}".format(state))
    # Set the state of the environment
    env.s = state
    # Used to hold all information for a single time step (including the image data)
    frames = []
    # Used to determine when the simulation has been completed
    done = False
    # Determines the number of times steps that the application has been run for
    time_steps = 0
    # The total values used to determine how many times the agent mistakenly
    # picks up no one or attempts to dropoff no passenger or attempts to
    # dropoff a passenger in the wrong position.
    penalties, reward = 0, 0
    # Run until the passenger has been picked up and dropped off in the target location
    while not done:
        # Perform a random action from the set of available actions in the environment
        action = env.action_space.sample()
        # From performing the action, retrieve the new state, the reward from taking the action,
        # whether the simulation is complete, and other information from performing the action.
        state, reward, done, info = env.step(action)
        # If an incorrect dropoff or pickup is performed, increment the penalty count
        if reward == -10:
            penalties += 1
        # Put each rendered frame into dict to use for animating the process and
        # tracking the details over the run
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        # Increment the time step count
        time_steps += 1
    # State the total number of steps taken and the total penalties that have occured.
    if not disable_prints:
        print("Timesteps taken: {}".format(time_steps))
        print("Penalties incurred: {}".format(penalties))
    # Return the frame data, the total penalties, and the total time steps
    return frames, penalties, time_steps

With the baseline approach defined, we will run a test with this approach to see how long it takes an agent using this approach to find a solution for simulation 328 and how many major penalties the agent receives.

In [ ]:
state = 328
# Run a test and collect all frames from the run
frames, _, _ = run_single_simulation_baseline(env, state)

Testing for simulation: 328
Timesteps taken: 3597
Penalties incurred: 1174


After performing a simulation and retrieving the results, we can use the frames obtained from the simulation and pass it to the *print_frames* function below to display an animation containing all frames along with the information within that frame at each timestep.    

For the first episode that you view, it is recommended to run through the entire process at a slower speed (such as 0.3 or 0.5 for the sleepTime parameter). However you are free to increase the speed of the process later on.

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, sleepTime=0.3):
    '''
    For each frame, show the frame and display the timestep it occurred at,
    the number of the active state, the action selected, adn the corresponding reward.
    '''
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        # Can adjust speed here
        sleep(sleepTime)


In [ ]:
# Print the frames from the episode
print_frames(frames, 0.1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 3597
State: 0
Action: 5
Reward: 20


**(TO DO) Q2 - 2 marks**   
Let's look at a few simulations to see how the random approach behaves.  To do so, we'll start from the states you've defined in Q1.

**(TO DO) Q2 (a) - 1 mark**   
Using the state defined from Q1 (a), retrieve the corresponding frames obtained from using the baseline approach defined above. Then display those frames.

In [ ]:
# ANSWER Q2(a)
# Retrieve the corresponding frames from running the simulation starting from the state found in Q1 (a). Then show those frames.
frames_1a, _, _ = run_single_simulation_baseline(env, state_1a)
print_frames(frames_1a, 0.1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 8676
State: 0
Action: 5
Reward: 20


**(TO DO) Q2 (b) - 1 mark**   
Using the state defined from Q1 (b), retrieve the corresponding frames obtained from using the baseline approach defined above. Then display those frames.

In [ ]:
# ANSWER Q2(b)
# Retrieve the corresponding frames from running the simulation starting from the state found in Q1 (a). Then show those frames.
frames_1b, _, _ = run_single_simulation_baseline(env, state_1b)
print_frames(frames_1b, 0.1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 22
State: 475
Action: 5
Reward: 20


With the ability to simulate single runs of an episode with the Baseline approach, we will now define a function that we will use to evaluate the general performance of the baseline model when averaged over many episodes. The *evaluate_agent_baseline* function below accepts as input the total number of randomly selected episodes to run along with the environment, runs the random episodes, displays the average amount of timesteps taken per episode along with the average penalties incurred, and returns the frame data.     



In [ ]:
def evaluate_agent_baseline(episodes, env):
    '''
    Given a number of episodes and an environment, run the specified
    number of episodes, where each run begins with a random state, display the
    naverage timesteps per episode and the average penalties per episode, and output
    the frames to be displayed.
    '''
    total_time_steps, total_penalties = 0, 0
    frames = []
    # Run through the total number of episodes
    for _ in range(episodes):
        # Get a random state
        state = env.reset()
        # Run the simulation, obtaining the results
        frame_data, penalties, time_steps = run_single_simulation_baseline(env, state, True)
        # Update the tracked data over all simulations
        total_penalties += penalties
        total_time_steps += time_steps
        frames = frames + frame_data
    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {total_time_steps / episodes}")
    print(f"Average penalties per episode: {total_penalties / episodes}")
    return frames

**(TO DO) Q3 - 5 marks**    
Let's evaluate the baseline approach.  We must run a certain number of episodes to average the results.

**(TO DO) Q3 (a) - 1 mark**    
Use the *evaluate_agent_baseline* function defined above to run through 100 random episodes for the environment.   

***If the evaluate_agent_baseline function ever seems to be running for far too long (several minutes, not just one), stop the run by clicking the button at the top-left of the code cell being executed and run it again.***

In [ ]:
# ANSWER Q3(a)
frames_3a = evaluate_agent_baseline(100, env)

Results after 100 episodes:
Average timesteps per episode: 2249.37
Average penalties per episode: 726.94


**(TO DO) Q3 (b) - 2 marks**    
From the output seen from Q3 (a), how did the Baseline approach do and why do you think that it performed well or poorly? Explain with respect to the average timesteps per episode and the average penalties per episode.     

**ANSWER Q3(b)**   

The Baseline approach performed poorly because the average timesteps per episode was 2249.37, and the average penalties per episode was 726.94. These are indicators of poor performance because that means it took a large number of timesteps per episode to get to the destination, which is not the desired outcome (to minimize the number of steps). Furthermore, the desired outcomes for penalties is zero or a value close to zero; thus, the large value for the average number of penalties from the simulation supports the fact that the Baseline approach performed poorly.  

I think the Baseline approach performed poorly because of the random nature of the algorithm, which selects a random available action at each step. So, a kind of brute force approach was used, which is not always the most optimal method.

**(TO DO) Q3 (c) - 2 marks**    
Without suggesting to use a Reinforcment Learning approach (as we would do next), suggest 2 ideas that could be included in a Baseline+ approach to allow it to perform slightly better?

**ANSWER Q3(c)**

Two other ideas that could help improve the Baseline approach would be a greedy strategy, with a (1) random restart algorithm or a (2) random modification algorithm. A greedy strategy is not the best solution since it tends to find a locally optimal solution. However, implementing that with a random restart can bring it closer to finding a globally optimal solution. The same applies for a random modification algorithm, since jumping to random places might lead to exploring a space toward a globally optimal solution. 


**3.0 - Training an Agent with Q-Learning to play the Taxi Game**   

Now that we have seen simulations of the autonomous taxi using a random strategy, we will use Q-Learning to try applying a Reinforcement Learning approach to the problem and have the autonomous taxi learn a better strategy.

To start the process, we will create a table of Q values for each action-state possibility (initializing it as zero). The agent will update this table when training and ATTENTION will need to ***reinitialize*** the table whenever the agent wants to restart its training.

In [ ]:
# Initialize the table of Q values for the state-action pairs
q_table = np.zeros([env.observation_space.n, env.action_space.n])

With the Q-table initialized, we will now define the training function that adjusts the Q values within *q_table*. The training process consists of running through a number of random simulations and updating the Q values for each state via Q-Learning.    

There are a number of hyperparameters used by the training function:    

- *alpha*: Learning parameter 
- *gamma*: The long term reward discount parameter.    
- *epsilon*: Exploitation/Exploration parameter 
- *num_simulations*: Represents how many episodes should be generated for the autonomous taxi update the Q-values.

Thus, by running through this learning algorithm, an agent can learn which Q-values to use when later put in a test situation.

In [ ]:
def train_agent(alpha, gamma, epsilon, num_simulations):
    '''
    Trains an agent by updating its Q values for a total of num_simulations
    episodes with the alpha, gamma, and epsilon hyperparameters. 
    '''
    # For plotting metrics
    all_time_steps = []
    all_penalties = []
    # Generate the specified number of episodes
    for i in range(1, num_simulations + 1):
        # Generate a new state by resetting it
        state = env.reset()
        # Variables tracked (time steps, total penalties, the reward value)
        time_steps, penalties, reward, = 0, 0, 0
        done = False
        # Run the simulation 
        while not done:
            # Select a random action is the randomly selected number from a
            # uniform distribution is less than epsilon
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            # Otherwise use the currently learned Q values
            else:
                action = np.argmax(q_table[state]) # Exploit learned values
            # Retrieve the relevant information after performing the action
            next_state, reward, done, info = env.step(action) 
            # Retrieve the old Q value and the maximum Q value from the next state
            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])
            # Update the current Q value
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value
            # Track anytime an incorrect dropoff or pickup is made
            if reward == -10:
                penalties += 1
            # Proceed to the next state and time step
            state = next_state
            time_steps += 1
        # Display progress for each 100 episodes
        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")
    print("Training finished.\n")

We now use the training function with a set of hyperparameters to train the agent with Q-Learning to potentially improve performance over time.  The number of episodes is set to 100000, so the learning will take a bit of time.

In [ ]:
# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.1
num_simulations = 100000
# Train the agent
train_agent(alpha, gamma, epsilon, num_simulations)

Episode: 100000
Training finished.



After the training, we can look at the Q-values that have been obtained in our state-action table for a specific state. Below we see that each Q-value for the six possible actions available for state 328 have been updated accordingly.

**(TO DO) Q4 - 2 marks**     
Below we print the Q-values that are available for the six actions at state 328 and we render that state to view it. Based on the available Q-values (assuming we are in exploitation mode), which action would be the next to be selected (or if there are ties, list all possible actions that would be considered)? Do any of the actions that contain larger Q values seem problematic if they were selected? Why or why not?

In [ ]:
print(q_table[328])
env.s = 328
env.render()

[ -1.98659605  -1.95703125  -1.98411132  -1.97592292 -10.25457278
 -10.50485063]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


**ANSWER Q4**

Assuming we are in exploitation mode, based on the available Q values, the action that would be selected next is *move north* (labelled as 1) with a Q value of -1.95703125.  
  
The problematic actions are pickup passenger (labelled as 4) with a Q value of -10.25457278, and drop off passenger (labelled as 5) with a Q value of -10.50485063. Because both these actions have very large Q values that far exceed the other actions, then the long-term reward would be poorly affected, so it would be quite problematic to choose either.  


With the training complete, we can now evaluate the Q-Learning approach in a similar method that we used to evaluate the Baseline approach. By passing the number of episodes to test for and the environment, we generate that number of random episodes and average the results obtained from running the Q-Learning approach to complete the episodes. Unlike the training, it is important to note that the hyperparameters that were used for learning are not used here. The agent simply uses the maximum Q-value at each step to determine which action to take at a given time step. This means we use a greedy policy (not even an epsilon-greedy one) which always trust the maximum Q-value to decide on its action.  It's a full exploitation mode (not exploration/exploitation).



In [ ]:
def evaluate_agent_QL(episodes, env):
    '''
    Given a number to specify how many random states to run and the environment to use,
    display the averaged metrics obtained from the tests and return the frames obtained from the tests.
    '''
    total_time_steps, total_penalties = 0, 0
    frames = []
    for _ in range(episodes):
        # Generate a random state to use
        state = env.reset()
        # The information collected throughout the run
        time_steps, penalties, reward = 0, 0, 0
        # Determines when the episode is complete
        done = False
        # Run through the episode until complete
        while not done:
            # Select the action containing the maximum Q value
            action = np.argmax(q_table[state])
            # Run that action and retrieve the reward and other details
            state, reward, done, info = env.step(action)

            # Put each rendered frame into dict for animation
            frames.append({
                'frame': env.render(mode='ansi'),
                'state': state,
                'action': action,
                'reward': reward
                }
            )
            # Specify whether the agent incorrectly chose to pick up or dropoff a passenger
            if reward == -10:
                penalties += 1
            # Increment the current time step
            time_steps += 1
        # Track the totals
        total_penalties += penalties
        total_time_steps += time_steps
    # Display the performance over the tests
    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {total_time_steps / episodes}")
    print(f"Average penalties per episode: {total_penalties / episodes}")
    # Return the frames to allow a user to view the runs
    return frames

**(TO DO) Q5 - 3 marks**     
 There is a random aspect in Q-learning, so we need to run for a certain number of episodes to provide an average result. Let's set that number high enough for the average to be significant. Let's see how Q-learning does compared to the baseline approach.

 ***ATTENTION: If the evaluate_agent_QL function ever seems to be running for far too long (30s or more), stop the run by clicking the button at the top-left of the code cell being executed and run it again. This occurs because the training was insufficient at setting valid Q values and resulted in a dead-end for a specific state.***

**(TO DO) Q5 (a) - 1 mark**     
Run the *evaluate_agent_QL* for 500 episodes to retrieve the average number of time steps and the average penalty after training.     

In [ ]:
# ANSWER Q5(a)
frames_5a = evaluate_agent_QL(500, env)


Results after 500 episodes:
Average timesteps per episode: 13.126
Average penalties per episode: 0.0


**(TO DO) Q5 (b) - 2 marks**     
Given your results from Q5 (a), how do the observed results from the tests compare to the tests from the Baseline model in Q3 (a)? Specifically, which policy performs better with respect to the average number of penalties throughout the tests and which strategy is able to take passengers from start to destination more rapidly (on average).     

**ANSWER Q5(b)**  
When comparing the observed results from the tests, it can be shown that the policy for Q5(a) performed significantly better than the policy for the Baseline model in Q3(a). For Q3(a), the average timesteps per episode is 2249.37, and for Q5(a), the average timesteps per episode is 13.126, which is a difference of 2236.244. Thus, Q5(a) has a more performant policy because it is able to take passengers from start to destination much more rapidly than the Baseline model in Q3(a). Furthermore, for Q3(a), the average penalties per episode is 726.94, and for Q5(a), the average penalties per episode is 0.0, which means the policy in Q5(a) performs better with respect to the average number of penalties (because its value is essentially 0).



**4.0 - Testing Different Hyperparameters**   

Now we will try retraining the agent using different set ups for the hyperparameters. This will allow you to explore their impact on the Q-Learning as well as understand their purpose during the training.     

**(TO DO) Q6 - 12 marks**     
Below we explore variations for all four hyperparameters used by the Q-Learning approach to better understand their impact on the training. When answering the questions, ***be careful to correctly set the hyperparameters***.         

As a note, below are the initial hyperparameter values used from section 3.0 of this notebook to use as reference:    

*alpha* = 0.1   
*gamma* = 0.5   
*epsilon* = 0.1   
*num_simulations* = 100000 

For all our evaluations, we will use 500 episodes.  Also, after each experiment, you will print the qtable value for state 328, so you can compare it with what was obtained in Q4 earlier.

***ATTENTION (repeated from Q5): If the evaluate_agent_QL function ever seems to be running for far too long (30s or more), stop the run by clicking the button at the top-left of the code cell being executed and run it again. This occurs because the training was insufficient at setting valid Q values and resulted in a dead-end for a specific state.***

**(TO DO) Q6 (a) - 1.5 marks**     
Retrain the agent by resetting the Q learning values and training for only **35000 episodes** (with the same alpha, gamma, and epsilon values used in section 3.0 of this notebook). Then perform another test for 500 episodes with the environment.  Print the *q_table* for state 328.

In [ ]:
# ANSWER Q6(a)
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.1
num_simulations = 35000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 500 episodes
frames_6a = evaluate_agent_QL(500, env)
# TODO: Print q_table for state 328
print(q_table[328])

Episode: 35000
Training finished.

Results after 500 episodes:
Average timesteps per episode: 13.148
Average penalties per episode: 0.0
[-1.97283402 -1.95703125 -1.97805076 -1.96125526 -8.88292347 -8.65427996]


**(TO DO) Q6 (b) - 1.5 marks**      
Retrain the agent by resetting the Q learning values and training for **100000 episodes**, but with an **epsilon value of 0.8** (with the same alpha and gamma values used in section 3.0 of this notebook). Then perform another test for 500 episodes with the environment.  Print the *q_table* for state 328.

In [ ]:
# ANSWER Q6(b)
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.8
num_simulations = 100000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 500 episodes
frames_6b = evaluate_agent_QL(500, env)
# TODO: Print q_table for state 328
print(q_table[328])

Episode: 100000
Training finished.

Results after 500 episodes:
Average timesteps per episode: 13.24
Average penalties per episode: 0.0
[ -1.98925781  -1.95703125  -1.98925781  -1.97851563 -10.97851563
 -10.97851563]


**(TO DO) Q6 (c) - 1.5 marks**      
Retrain the agent by resetting the Q learning values and training for **100000 episodes**, but with an **alpha value of 0.7** (with the same gamma and epsilon values used in section 3.0 of this notebook). Then perform another test for 500 episodes with the environment.    Print the *q_table* for state 328. 

In [ ]:
# ANSWER Q6(c)
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.7
gamma = 0.5
epsilon = 0.1
num_simulations = 100000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 500 episodes
frames_6c = evaluate_agent_QL(500, env)
# TODO: Print q_table for state 328
print(q_table[328])

Episode: 100000
Training finished.

Results after 500 episodes:
Average timesteps per episode: 13.114
Average penalties per episode: 0.0
[ -1.98925781  -1.95703125  -1.98925781  -1.97851562 -10.97851562
 -10.97851562]


**(TO DO) Q6 (d) - 1.5 marks**      
Retrain the agent by resetting the Q learning values and training for **100000 episodes**, but with an **gamma value of 0.15** (with the same alpha and epsilon values used in section 3.0 of this notebook). Then perform another test for 500 episodes with the environment.  Print the *q_table* for state 328.  

In [ ]:
# ANSWER Q6(d)
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.1
gamma = 0.15
epsilon = 0.1
num_simulations = 100000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 500 episodes
frames_6d = evaluate_agent_QL(500, env)
# TODO: Print q_table for state 328
print(q_table[328])

Episode: 100000
Training finished.

Results after 500 episodes:
Average timesteps per episode: 13.346
Average penalties per episode: 0.0
[-1.17647045 -1.17646977 -1.17647049 -1.17647037 -9.94596458 -9.99010609]


**(TO DO) Q6 (e) - 6 marks**      
Using the results obtained from your tests in Q6 (a), (b), (c), and (d), along with the initial results found from Q5 to serve as the Q-Learning baseline to compare with, explain the impacts of modifying the number of episodes trained on (less vs more), the alpha value (lower vs higher), the gamma value (lower vs higher), and the epsilon value (lower vs higher).  Even if the difference in the comparisons are minor, state them.  

Besides looking at the obtained results (average number of timesteps), discuss the impact on the q-values for the different actions in state 328. Did those change?


**ANSWER Q6(e)**  

Make sure you discuss each hyperparameter in relation to timesteps and qvalues  
compare with, explain the impacts of modifying  

**Number of Episodes trained on: Less vs. More**  
To compare the number of episodes trained on, we will look at the results obtained from Q5 and the results obtained from Q6(a) because the only major change between these two models is the number of episodes trained on. Q5 trained on 100000 episodes and Q6(a) trained on 35000 episodes, which is a difference of 65000. The impact of reducing the number of episodes trained seemed to produce a slightly less optimal result for the average timesteps per episode. The average timesteps per episode for Q5 is 13.126, but for Q6(a), it is 13.148; thus, in this case, a reduction in 65000 episodes trained on has increased the average timesteps per episode by 0.022. Since we want a small number of timesteps, it appears that having less episodes trained on will reduce the performance in terms of average timesteps per episode (though by a small margin). Thus, it is better to have more episodes trained on if we want to improve performance.  

To make things easier to visualize, here are the Q-Values for:   
Q5: [-1.98659605, -1.95703125, -1.98411132, -1.97592292, -10.25457278, -10.50485063]  
Q6(a): [-1.97283402, -1.95703125, -1.97805076, -1.96125526, -8.88292347, -8.65427996]   

When looking at the Q-Values, it seems that the reduction of the *number of episodes trained on* has an impact for state 328, but what is not affected is the action to be selected next. In other words, both the policy in Q5 and Q6a will choose *move north* as the next action for state 328. However, for state 328, most of the Q-values for 6(a) appear to be slightly larger than those for Q5, apart from the Q-value for *move up* (at index 1), which is identical for both. The largest difference appears at index 4 and index 5, for actions *pickup passenger* and *drop off passenger*, respectively. For Q6(a), index 4 increased to -8.88292347 from -10.25457278 in Q5, which is a difference of approximately 1.37, and index 5 increased by approximately 1.85. Other increases were very minimal. For example, the q-values at index 0 (*move south*), index 2 (*move east*), and index 3 (*move west*) did not increase by more than approximately 0.015.
  

**Epsilon Value: Lower vs Higher**  
To analyze the epsilon value, we will compare the results from Q5 to the results in Q6(b) because the only notable change between these two models is the epsilon value. Q5 has an epsilon value of 0.1 and Q6(b) has an epsilon value of 0.8, which is a difference of 0.7. It appears that increasing the epsilon value will poorly impact the performance for the average timesteps per episode. The results support this claim because the average timesteps per episode for Q6(b) is 13.24 and for Q5 is 13.126, that means increasing the epsilon value by 0.7 has increased the average timesteps per episode by 0.114 (in this context). Thus, a higher epsilon value will yield a higher value for average timesteps per episode, which is a decrease in performance. Conversely, a lower epsilon value will yield a lower average timesteps per episode, which means an increase in performance.  

To make things easier to visualize, here are the Q-Values for:   
Q5: [-1.98659605, -1.95703125, -1.98411132, -1.97592292, -10.25457278, -10.50485063]
Q6(b): [-1.98925781, -1.95703125, -1.98925781, -1.97851563, -10.97851563, -10.97851563]  

When looking at the Q-Values, it seems that increasing the epsilon value will have an impact for state 328. All the Q-Values in Q6(b) seem to have a very small decrease from those in Q5, except for *move north*, which has an identical Q-value of -1.95703125 for both Q5 and Q6(b). It is interesting to note that the action to be selected next is not affected; both Q5 and Q6(b) will select *move north* as the next action for state 328. The majority of decreases from Q5 to Q6(b) were very small and less than approximately 0.01. The largest of these decreases were seen at index 4 (*pickup passenger*) from -10.25457278 in Q5 to  -10.97851563 in Q6(b), which is a difference of approximately 0.724. No other actions for state 328 surpassed this q-value when we refer to the increase.

  
**Alpha Value: Lower vs Higher**  
To analyze the alpha value, we will look at the results taken from Q5 and compare those to the results from Q6(c) because the only significant modification between these two models is the alpha value. Q5 has an alpha value of 0.1 and Q6(c) has an alpha value of 0.7, which is a difference of 0.6. The average timesteps per episode for Q5 is 13.126 and the average timesteps per episode for Q6(c) is 13.114, which is a difference of 0.012. For this situation, we can see that an increase in the alpha value by 0.6 has lead to a decrease in the average timesteps per episode by 0.012, which is desirable since we want to minimize the average number of timesteps. Thus, based on the results, we can state that a higher alpha value improves performance because it reduces the average timesteps per episode. We can also state that a lower alpha value will lead to worse performance because it will increase the average timesteps per episode.  
  
To make things easier to visualize, here are the Q-Values for:   
Q5: [-1.98659605, -1.95703125, -1.98411132, -1.97592292, -10.25457278, -10.50485063]  
Q6(c): [ -1.98925781  -1.95703125  -1.98925781  -1.97851562 -10.97851562
 -10.97851562]   
  
When looking at the Q-Values, it seems that increasing the alpha value will have an impact for state 328. All the Q-Values in Q6(c) seem to have a very small decrease from those in Q5, except for *move north*, which has an identical Q-value of -1.95703125 for both Q5 and Q6(c). Once again, it is interesting to see that the action to be selected next is not affected; both Q5 and Q6(c) will select *move north* as the next action for state 328. Once again, the largest differences can be seen at index 4 and 5, for the actions *pickup passenger* and *drop off passenger*, respectively.


**Gamma Value: Lower vs Higher**  
To analyze the gamma value, we will compare the results from Q5 to those in Q6(d) because the only notable change between these two models is the gamma value. Q5 has an gamma value of 0.5 and Q6(d) has an gamma value of 0.15, which is a difference of 0.35. The average timesteps per episode for Q5 is 13.126 and the average timesteps per episode for Q6(d) is 13.346, which is a difference of 0.22. In this context, the results show that decreasing the gamma value by 0.35 has increased the average timesteps per episode by 0.22. Thus, based on the results, a lower gamma value equates a higher average timesteps per episode, which is a reduction in performance. Conversely, we can say that a higher gamma value will improve performance, since it will decrease the average timesteps per episode.  
  
To make things easier to visualize, here are the Q-Values for:   
Q5: [-1.98659605, -1.95703125, -1.98411132, -1.97592292, -10.25457278, -10.50485063]  
Q6(d): [-1.17647045 -1.17646977 -1.17647049 -1.17647037 -9.94596458 -9.99010609]  

When looking at the Q-Values for state 328, it seems that decreasing (or adjusting) the gamma value has the largest impact than adjusting any of the other values (alpha, epsilon) or the *number of episodes trained on*. Decreasing the gamma value is the only time where we see a difference for the action *move north* from Q5 to Q6(d). Although move north* is still selected as next action, we see a change in its Q-value for the first time, with an increase of approximately 0.781 from Q5 to Q6(d). The rest of the differences for the Q-values approximate around this value. However, index 4 (*pickup passenger*) and index 5 (*drop off passenger*) show the largest differences again, at approximately 0.31 and 0.52, respectively.  

All in all, I also want to state that I did not place much emphasis for *the average penalties per episode* in my comparisons because they all had the same value of 0.0 across all tests.

**5.0 - Scaling up the environment**   

As we've seen in the video lectures, and as you've experienced here, a limitation of Q-Learning is that Q-Values must be learned for all state/action pairs.  In our toy problem of the autonomous taxi, we worked with a small grid (5x5) and only 4 pick-up/drop-off locations, and we were already at 500 states, so with 6 actions, that meant 3000 state/action pairs, so 3000 q-values to learn.

Imagine a larger grid of 100x100, and imagine all grid cells could be pick-up or drop-off cells, then Q-learning would not be a realistic approach and we would have to go to Deep Q-Learning, as we saw in the course videos.

Implementing a Deep Q-Learning approach is beyond the scope of this notebook, but we can still think about the type of architecture it would require.

**(TO DO) Q7 - 2 marks**      
Given this new description of a larger grid, and a larger set of pick-up/drop-off locations, describe what a deep Q-learning neural network could look like.  What could be put at the input nodes and how many would be required?  What would we try to learn at the output nodes and how many output nodes would be required.?

**ANSWER Q7**

The input nodes could be all the possible states.
From the output nodes, we would try to learn the best action given a state.
We would have as many output nodes as we have actions. So we would have 6 output nodes, a node for *move south*, *move north*, *move east*, *move west*, *pickup passenger*, and *drop off passenger*. 